Syntax natürlicher Sprachen, WS 2023/24

# 11 - Übung (Lösung)

In [1]:
import nltk
from nltk import Tree
from nltk.grammar import ProbabilisticProduction, PCFG

## Aufgabe 1 - Lexikalisierte CFG

#### Gegeben sei folgende FCFG (für einen Satz mit mit adverbialem PP-Komplement) mit nur teilweise durchgeführter Kopfannotation über ein `HEAD`-Merkmal.

#### Vervollständigen Sie die Kopfannotation gemäß UD-Regeln.

In [2]:
sentence = "sie stellt die Blumen in die Vase"

gramstring = r"""
% start S
    S[HEAD=?x] -> NP[] VP[HEAD=?x]
    NP[HEAD=?x] -> PRON[HEAD=?x]
    NP[HEAD=?x] -> DET[] N[HEAD=?x]
    NP[HEAD=?x] -> NP[HEAD=?x] PP[]
    VP[HEAD=?x] -> V[HEAD=?x] NP[] PP[]
    PP[HEAD=?x] -> P[] NP[HEAD=?x]
    
    PRON[HEAD="sie"] -> "sie"
    DET[HEAD="die"] -> "die"
    N[HEAD="Blume"] -> "Blumen"
    N[HEAD="Vase"] -> "Vase"
    V[HEAD="stellen"] -> "stellt"
    P[HEAD="in"] -> "in"
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    print(tree)
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #display(tree)

(S[HEAD='stellen']
  (NP[HEAD='sie'] (PRON[HEAD='sie'] sie))
  (VP[HEAD='stellen']
    (V[HEAD='stellen'] stellt)
    (NP[HEAD='Blume'] (DET[HEAD='die'] die) (N[HEAD='Blume'] Blumen))
    (PP[HEAD='Vase']
      (P[HEAD='in'] in)
      (NP[HEAD='Vase'] (DET[HEAD='die'] die) (N[HEAD='Vase'] Vase)))))
                 S[HEAD='stellen']                                                                                                                
       ┌─────────────────┴──────────────────────────────────────────────────┐                                                                      
       │                                                            VP[HEAD='stellen']                                                            
       │                 ┌────────────────────────────────┬─────────────────┴──────────────────────────────┐                                       
       │                 │                                │                                         PP[HEAD='V

## Aufgabe 2 - Parent Annotation

#### Gegeben sei folgende CFG mit unvollständiger *Parent Annotation*.

### a) Vervollständigen Sie über Symbolerweiterung (mit `^` als Trennerzeichen) die *Parent Annotation*, wie sie durch die Regelanwendungen im Syntaxbaum der Angabe impliziert ist.

#### Führen Sie außerdem eine *Grandparent Annotation* durch.


In [3]:
#Parent Annotation:
sentence = "sie sieht die Frau"

grammar = nltk.CFG.fromstring("""
    S   -> NP^S VP^S
    VP^S  -> V^VP NP^VP
    NP^VP  -> DET^NP N^NP
    NP^S  -> PRON^NP

    DET^NP -> "die"
    N^NP      -> "Frau"
    PRON^NP   -> "sie"
    V^VP      -> "sieht"
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

               S                   
   ┌───────────┴────┐               
   │               VP^S            
   │      ┌─────────┴──────┐        
  NP^S    │              NP^VP     
   │      │         ┌──────┴────┐   
PRON^NP  V^VP     DET^NP       N^NP
   │      │         │           │   
  sie   sieht      die         Frau



In [4]:
#Grandparent Annotation:
sentence = "sie sieht die Frau"

grammar = nltk.CFG.fromstring("""
    S   -> NP^S VP^S
    VP^S  -> V^VP^S NP^VP^S
    NP^VP^S  -> DET^NP^VP N^NP^VP
    NP^S  -> PRON^NP^S

    DET^NP^VP -> "die"
    N^NP^VP      -> "Frau"
    PRON^NP^S   -> "sie"
    V^VP^S      -> "sieht"
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

                  S                           
    ┌─────────────┴──────┐                     
    │                   VP^S                  
    │       ┌────────────┴────────┐            
   NP^S     │                  NP^VP^S        
    │       │            ┌────────┴───────┐    
PRON^NP^S V^VP^S     DET^NP^VP         N^NP^VP
    │       │            │                │    
   sie    sieht         die              Frau 



### b) Vergleichen Sie Ihre Lösung anschließend mit dem Resultat der entsprechenden Normalisierung mit NLTK (Chomsky-Normalform-Binarisierung mit Grandparent-Annotation):

In [5]:
#parent annotation:
tree = nltk.Tree.fromstring("""
(S
   (NP (PRON sie))
   (VP
    (V sieht) 
    (NP (DET die) (N Frau))
    ))
""")

#tree_copy = copy.deepcopy(tree)
tree.chomsky_normal_form(vertMarkov=1) 
tree.pretty_print(unicodelines=True)

              S                     
  ┌───────────┴────┐                 
  │              VP^<S>             
  │      ┌─────────┴───────┐         
NP^<S>   │              NP^<VP>     
  │      │         ┌───────┴─────┐   
 PRON    V        DET            N  
  │      │         │             │   
 sie   sieht      die           Frau



In [6]:
#grandparent annotation:
tree = nltk.Tree.fromstring("""
(S
   (NP (PRON sie))
   (VP
    (V sieht) 
    (NP (DET die) (N Frau))
    ))
""")

#tree_copy = copy.deepcopy(tree)
tree.chomsky_normal_form(vertMarkov=2) 
tree.pretty_print(unicodelines=True)

              S                       
  ┌───────────┴────┐                   
  │              VP^<S>               
  │      ┌─────────┴────────┐          
NP^<S>   │              NP^<VP-S>     
  │      │         ┌────────┴──────┐   
 PRON    V        DET              N  
  │      │         │               │   
 sie   sieht      die             Frau



### c) Vergleichen Sie auch das Ergebnis der entsprechende Normalisierung mit folgender Variante des Syntaxbaums mit flacher Struktur ohne VP:

In [7]:
tree = nltk.Tree.fromstring("""
(S
   (NP (PRON sie))
   (V sieht) 
   (NP (DET die) (N Frau))
    )
""")
tree.pretty_print(unicodelines=True)

       S               
 ┌─────┼────────┐       
 NP    │        NP     
 │     │    ┌───┴───┐   
PRON   V   DET      N  
 │     │    │       │   
sie  sieht die     Frau



In [8]:
#parent annotation und binarization:
tree.chomsky_normal_form(vertMarkov=1) 
tree.pretty_print(unicodelines=True)

              S                      
  ┌───────────┴─────┐                 
  │              S|<V-NP>            
  │      ┌──────────┴───────┐         
NP^<S>   │                NP^<S>     
  │      │          ┌───────┴─────┐   
 PRON    V         DET            N  
  │      │          │             │   
 sie   sieht       die           Frau



## *Aufgabe 3 - Grammatikinduktion aus Penn Treebank*

#### In dieser Aufgabe soll vollautomatisch aus Daten (Syntaxbäumen) eine probabilistische kontextfreie Grammatik erzeugt werden.

###  a) Herunterladen von Ressourcen und Beispiel

#### Laden Sie zunächst die Ressource „corpora/treebank“ mithilfe des NLTK-Download-Managers herunter, falls diese noch nicht installiert ist.

In [9]:
# nltk.download()

 #### Hier ein Beispiel für geparste Sätze in der Penn Treebank:

In [10]:
from nltk.corpus import treebank

for tree in treebank.parsed_sents('wsj_0001.mrg'):
    print(tree)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))
(S
  (NP-SBJ (NNP Mr.) (NNP Vinken))
  (VP
    (VBZ is)
    (NP-PRD
      (NP (NN chairman))
      (PP
        (IN of)
        (NP
          (NP (NNP Elsevier) (NNP N.V.))
          (, ,)
          (NP (DT the) (NNP Dutch) (VBG publishing) (NN group))))))
  (. .))


#### Hier ein Beispiel für Gramatikinduktion mit `nltk.induce_pcfg`:

In [11]:
productions = []
S = nltk.Nonterminal('S')
for tree in nltk.corpus.treebank.parsed_sents('wsj_0001.mrg'):
    productions += tree.productions()
len(productions)

51

In [12]:
grammar = nltk.induce_pcfg(S, productions)

print(len(grammar.productions()))
for production in grammar.productions():
    print(production)

44
S -> NP-SBJ VP . [1.0]
NP-SBJ -> NP , ADJP , [0.5]
NP -> NNP NNP [0.25]
NNP -> 'Pierre' [0.125]
NNP -> 'Vinken' [0.25]
, -> ',' [1.0]
ADJP -> NP JJ [1.0]
NP -> CD NNS [0.125]
CD -> '61' [0.5]
NNS -> 'years' [1.0]
JJ -> 'old' [0.5]
VP -> MD VP [0.333333]
MD -> 'will' [1.0]
VP -> VB NP PP-CLR NP-TMP [0.333333]
VB -> 'join' [1.0]
NP -> DT NN [0.125]
DT -> 'the' [0.666667]
NN -> 'board' [0.25]
PP-CLR -> IN NP [1.0]
IN -> 'as' [0.5]
NP -> DT JJ NN [0.125]
DT -> 'a' [0.333333]
JJ -> 'nonexecutive' [0.5]
NN -> 'director' [0.25]
NP-TMP -> NNP CD [1.0]
NNP -> 'Nov.' [0.125]
CD -> '29' [0.5]
. -> '.' [1.0]
NP-SBJ -> NNP NNP [0.5]
NNP -> 'Mr.' [0.125]
VP -> VBZ NP-PRD [0.333333]
VBZ -> 'is' [1.0]
NP-PRD -> NP PP [1.0]
NP -> NN [0.125]
NN -> 'chairman' [0.25]
PP -> IN NP [1.0]
IN -> 'of' [0.5]
NP -> NP , NP [0.125]
NNP -> 'Elsevier' [0.125]
NNP -> 'N.V.' [0.125]
NP -> DT NNP VBG NN [0.125]
NNP -> 'Dutch' [0.125]
VBG -> 'publishing' [1.0]
NN -> 'group' [0.25]


#### Beispiel-Parse mit induzierter Grammatik:

In [13]:
parser = nltk.ViterbiParser(grammar)
sent = "Mr. Pierre is the director of Elsevier N.V. .".split()
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP-SBJ (NNP Mr.) (NNP Pierre))
  (VP
    (VBZ is)
    (NP-PRD
      (NP (DT the) (NN director))
      (PP (IN of) (NP (NNP Elsevier) (NNP N.V.)))))
  (. .)) (p=1.05964e-07)
                                 S                                 
      ┌──────────────────────────┼───────────────────────────────┐  
      │                          VP                              │ 
      │            ┌─────────────┴───────┐                       │  
      │            │                   NP-PRD                    │ 
      │            │       ┌─────────────┴───────┐               │  
      │            │       │                     PP              │ 
      │            │       │             ┌───────┴──────┐        │  
    NP-SBJ         │       NP            │              NP       │ 
 ┌────┴──────┐     │   ┌───┴─────┐       │       ┌──────┴───┐    │  
NNP         NNP   VBZ  DT        NN      IN     NNP        NNP   . 
 │           │     │   │         │       │       │          │    │  

### b) Induktion von PCFG-Regeln aus der Penn-Treebank

#### Im Folgenden wollen wir  - analog zur `induce_pcfg`-Methode des NLTK - vollautomatisch eine aus den Syntaxbäumen der Penn Treebank induzierte Grammatik erzeugen.

#### Füllen Sie die Lücken im untenstehenden Code und versuchen Sie, mit Hilfe der automatisch erstellten PCFG die folgenden Sätze zu parsen:

In [16]:
test_sentences = [
    "the men saw a car .",
    "the woman gave the man a book .",
    "she gave a book to the man .",
    "yesterday , all my trouble seemed so far away ."
]

In [17]:
from collections import defaultdict

# Production count: the number of times a given production occurs
pcount = defaultdict(int)

# LHS-count: counts the number of times a given lhs occurs
lcount = defaultdict(int)

for tree in treebank.parsed_sents():
    for prod in tree.productions():
        pcount[prod] += 1
        lcount[prod.lhs()] += 1
        
productions = [
    ProbabilisticProduction(
        p.lhs(), p.rhs(),
        prob = pcount[p] / lcount[p.lhs()]
    )
    for p in pcount
]

start = nltk.Nonterminal('S')
grammar = PCFG(start, productions)
parser = nltk.ViterbiParser(grammar)

In [18]:
for s in test_sentences:
    for t in parser.parse(nltk.word_tokenize(s)):
        print(t.prob())
        t.pretty_print(unicodelines=True)

2.269940263066198e-15
                S                 
      ┌─────────┴───┬───────────┐  
      │             VP          │ 
      │         ┌───┴───┐       │  
    NP-SBJ      │       NP      │ 
 ┌────┴─────┐   │   ┌───┴───┐   │  
 DT        NNS VBD  DT      NN  . 
 │          │   │   │       │   │  
the        men saw  a      car  . 

8.44395440336237e-21
                       S                             
      ┌────────────────┴───────┬───────────────────┐  
      │                        VP                  │ 
      │           ┌────────┬───┴────────┐          │  
    NP-SBJ        │        NP         NP-TMP       │ 
 ┌────┴──────┐    │    ┌───┴───┐   ┌────┴─────┐    │  
 DT          NN  VBD   DT      NN  DT         NN   . 
 │           │    │    │       │   │          │    │  
the        woman gave the     man  a         book  . 

1.3157826588159793e-18
                     S                         
  ┌──────────────────┼───────────────────────┐  
  │                  VP   

## Aufgabe 4 - Disambiguierung mit PCFG

### a) Gegeben sei folgende Mini-Treebank mit PP-Attachment-Sätzen:

In [19]:
treestrings = [
"""(S
   (NP (PRON sie))
   (VP
    (V kennt) 
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas))))))""",

"""(S
   (NP (PRON sie))
   (VP
    (VP (V beobachtet) (NP (DET den) (N Mann)))
    (PP (P mit) (NP (DET dem) (N Fernglas)))))""",


"""(S
   (NP (PRON sie))
   (VP
    (VP (V stellt) (NP (DET die) (N Blumen)))
    (PP (P in) (NP (DET die) (N Vase)))))""",

]

trees = [Tree.fromstring(treestring).pretty_print(unicodelines=True) for treestring in treestrings]


       S                                    
 ┌─────┴────────────┐                        
 │                  VP                      
 │     ┌────────────┴────┐                   
 │     │                 NP                 
 │     │        ┌────────┴───┐               
 │     │        │            PP             
 │     │        │        ┌───┴───┐           
 NP    │        NP       │       NP         
 │     │    ┌───┴───┐    │   ┌───┴─────┐     
PRON   V   DET      N    P  DET        N    
 │     │    │       │    │   │         │     
sie  kennt den     Mann mit dem     Fernglas

         S                                       
 ┌───────┴───────────────┐                        
 │                       VP                      
 │               ┌───────┴────────┐               
 │               VP               PP             
 │       ┌───────┴───┐        ┌───┴───┐           
 NP      │           NP       │       NP         
 │       │       ┌───┴───┐    │   ┌───┴─────┐     
PRON

#### Passen Sie diese Mini-Treebank durch Duplizieren einer minimalen Anzahl an Sätzen so an, dass die daraus induzierte Grammatik für den ersten Satz eine korrekte Analyse liefert (so dass die Grammatik also das PP-Attachment an die NP bevorzugt).

In [20]:
treestrings = [
"""(S
   (NP (PRON sie))
   (VP
    (V kennt) 
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas))))))""", #NP-Attachment

"""(S
   (NP (PRON sie))
   (VP
    (V kennt) 
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas))))))""", #NP-Attachment (dupliziert)

"""(S
   (NP (PRON sie))
   (VP
    (V kennt) 
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas))))))""", #NP-Attachment (dupliziert)

"""(S
   (NP (PRON sie))
   (VP
    (V kennt) 
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas))))))""", #NP-Attachment (dupliziert)

"""(S
   (NP (PRON sie))
   (VP
    (V kennt) 
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas))))))""", #NP-Attachment (dupliziert)

"""(S
   (NP (PRON sie))
   (VP
    (V kennt) 
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas))))))""", #NP-Attachment (dupliziert)

"""(S
   (NP (PRON sie))
   (VP
    (V kennt) 
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas))))))""", #NP-Attachment (dupliziert)




"""(S
   (NP (PRON sie))
   (VP
    (VP (V beobachtet) (NP (DET den) (N Mann)))
     (PP (P mit) (NP (DET dem) (N Fernglas)))))""", #VP-Attachment


"""(S
   (NP (PRON sie))
   (VP
    (VP (V stellt) (NP (DET die) (N Blumen)))
    (PP (P in) (NP (DET die) (N Vase)))))""", #VP-Attachment

]


trees = [Tree.fromstring(treestring) for treestring in treestrings]

#grammar induction:    
productions = []
S = nltk.Nonterminal('S')

for tree in trees:
    productions += tree.productions()

grammar = nltk.induce_pcfg(S, productions)
for production in grammar.productions():
    print(production)    

#parse first tree with grammar:
parser = nltk.InsideChartParser(grammar)
for parse in parser.parse(trees[0].leaves()):
    print(parse)
    parse.pretty_print(unicodelines=True)

S -> NP VP [1.0]
NP -> PRON [0.264706]
PRON -> 'sie' [1.0]
VP -> V NP [0.818182]
V -> 'kennt' [0.777778]
NP -> NP PP [0.205882]
NP -> DET N [0.529412]
DET -> 'den' [0.444444]
N -> 'Mann' [0.444444]
PP -> P NP [1.0]
P -> 'mit' [0.888889]
DET -> 'dem' [0.444444]
N -> 'Fernglas' [0.444444]
VP -> VP PP [0.181818]
V -> 'beobachtet' [0.111111]
V -> 'stellt' [0.111111]
DET -> 'die' [0.111111]
N -> 'Blumen' [0.0555556]
P -> 'in' [0.111111]
N -> 'Vase' [0.0555556]
(S
  (NP (PRON sie))
  (VP
    (V kennt)
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas)))))) (p=0.000337126)
       S                                    
 ┌─────┴────────────┐                        
 │                  VP                      
 │     ┌────────────┴────┐                   
 │     │                 NP                 
 │     │        ┌────────┴───┐               
 │     │        │            PP             
 │     │        │        ┌───┴───┐           
 NP    │        NP       │     

#### Parsen Sie abschließend die Sätze mit der induzierten Grammatik:

In [21]:
#parse trees with grammar:
parser = nltk.ViterbiParser(grammar)
for tree in trees:
    for parse in parser.parse(tree.leaves()):
        #print(parse)
        parse.pretty_print(unicodelines=True)

       S                                    
 ┌─────┴────────────┐                        
 │                  VP                      
 │     ┌────────────┴────┐                   
 │     │                 NP                 
 │     │        ┌────────┴───┐               
 │     │        │            PP             
 │     │        │        ┌───┴───┐           
 NP    │        NP       │       NP         
 │     │    ┌───┴───┐    │   ┌───┴─────┐     
PRON   V   DET      N    P  DET        N    
 │     │    │       │    │   │         │     
sie  kennt den     Mann mit dem     Fernglas

       S                                    
 ┌─────┴────────────┐                        
 │                  VP                      
 │     ┌────────────┴────┐                   
 │     │                 NP                 
 │     │        ┌────────┴───┐               
 │     │        │            PP             
 │     │        │        ┌───┴───┐           
 NP    │        NP       │       NP         

### b) Verbesserung durch Lexikalisierung

#### Führen Sie in der ursprünglichen Mini-Treebank eine teilweise Lexikalisierung durch Kopfannotation durch, so dass je nach Verb die entsprechende Konstruktion bevorzugt wird.

In [22]:
treestrings = [
"""(S
   (NP (PRON sie))
   (VP_kennt
    (V_kennt kennt) 
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas))))))""", #NP-Attachment

"""(S
   (NP (PRON sie))
   (VP
    (VP_beobachtet (V_beobachtet beobachtet) (NP (DET den) (N Mann)))
     (PP (P mit) (NP (DET dem) (N Fernglas)))))""", #VP-Attachment


"""(S
   (NP (PRON sie))
   (VP
    (VP_stellt (V_stellt stellt) (NP (DET die) (N Blumen)))
    (PP (P in) (NP (DET die) (N Vase)))))""", #VP-Attachment

]


trees = [Tree.fromstring(treestring) for treestring in treestrings]
# trees = []
# for treestring in treestrings:
#     trees.append(Tree.fromstring(treestring))

#grammar induction:    
productions = []
S = nltk.Nonterminal('S')

for tree in trees:
    productions += tree.productions()

grammar = nltk.induce_pcfg(S, productions)
for production in grammar.productions():
    print(production)    

#parse trees with grammar:
parser = nltk.ViterbiParser(grammar)
for tree in trees:
    for parse in parser.parse(tree.leaves()):
        print(parse)
        parse.pretty_print(unicodelines=True)

S -> NP VP_kennt [0.333333]
NP -> PRON [0.3]
PRON -> 'sie' [1.0]
VP_kennt -> V_kennt NP [1.0]
V_kennt -> 'kennt' [1.0]
NP -> NP PP [0.1]
NP -> DET N [0.6]
DET -> 'den' [0.333333]
N -> 'Mann' [0.333333]
PP -> P NP [1.0]
P -> 'mit' [0.666667]
DET -> 'dem' [0.333333]
N -> 'Fernglas' [0.333333]
S -> NP VP [0.666667]
VP -> VP_beobachtet PP [0.5]
VP_beobachtet -> V_beobachtet NP [1.0]
V_beobachtet -> 'beobachtet' [1.0]
VP -> VP_stellt PP [0.5]
VP_stellt -> V_stellt NP [1.0]
V_stellt -> 'stellt' [1.0]
DET -> 'die' [0.333333]
N -> 'Blumen' [0.166667]
P -> 'in' [0.333333]
N -> 'Vase' [0.166667]
(S
  (NP (PRON sie))
  (VP_kennt
    (V_kennt kennt)
    (NP
      (NP (DET den) (N Mann))
      (PP (P mit) (NP (DET dem) (N Fernglas)))))) (p=2.96296e-05)
        S                                         
 ┌──────┴───────────────┐                          
 │                   VP_kennt                     
 │      ┌───────────────┴──────┐                   
 │      │                      NP           

### c) Verbesserung durch Parent-Annotation

#### Folgend Mini-Treebank enthält Sätze mit unterschiedlicher Anzahl pronominaler bzw. nominaler NPs unter S-Knoten.

#### Warum sind in der induzierten Grammatik die Ableitungswahrscheinlichkeiten gleich, obwohl eine Konstruktion überwiegt?

#### Führen Sie eine teilweise Parent Annotation durch um ein adäquateres Modell zu bekommen (d.h. die Wahrscheinlichkeit für die letzte Ableitung mit nominaler NP unter S-Knoten soll kleiner als die für die anderen werden).

In [ ]:
# Antwort:
# beide Ableitungstypen verwenden die selben Regeln, nur andere Position.

In [23]:
treestrings = [
"""(S
   (NP^S (PRON sie))
   (VP
    (V sieht) 
    (NP^VP (DET die) (N Frau))
    ))
""",

"""(S
   (NP^S (PRON sie))
   (VP
    (V sieht) 
    (NP^VP (DET die) (N Frau))
    ))
""",


"""(S
    (NP^S (DET die) (N Frau))
   (VP
    (V sieht) 
   (NP^VP (PRON sie))
    ))
""",

]


trees = [Tree.fromstring(treestring) for treestring in treestrings]

#grammar induction:    
productions = []
S = nltk.Nonterminal('S')

for tree in trees:
    productions += tree.productions()

grammar = nltk.induce_pcfg(S, productions)
for production in grammar.productions():
    print(production)    

#parse trees with grammar:
parser = nltk.ViterbiParser(grammar)
for tree in trees:
    for parse in parser.parse(tree.leaves()):
        print(parse)
        parse.pretty_print(unicodelines=True)

S -> NP^S VP [1.0]
NP^S -> PRON [0.666667]
PRON -> 'sie' [1.0]
VP -> V NP^VP [1.0]
V -> 'sieht' [1.0]
NP^VP -> DET N [0.666667]
DET -> 'die' [1.0]
N -> 'Frau' [1.0]
NP^S -> DET N [0.333333]
NP^VP -> PRON [0.333333]
(S
  (NP^S (PRON sie))
  (VP (V sieht) (NP^VP (DET die) (N Frau)))) (p=0.444444)
            S                
 ┌──────────┴───┐             
 │              VP           
 │     ┌────────┴────┐        
NP^S   │           NP^VP     
 │     │        ┌────┴────┐   
PRON   V       DET        N  
 │     │        │         │   
sie  sieht     die       Frau

(S
  (NP^S (PRON sie))
  (VP (V sieht) (NP^VP (DET die) (N Frau)))) (p=0.444444)
            S                
 ┌──────────┴───┐             
 │              VP           
 │     ┌────────┴────┐        
NP^S   │           NP^VP     
 │     │        ┌────┴────┐   
PRON   V       DET        N  
 │     │        │         │   
sie  sieht     die       Frau

(S
  (NP^S (DET die) (N Frau))
  (VP (V sieht) (NP^VP (PRON sie)))) (p=0.